# Tutorial: Natural Language Inference

In this short tutorial, we show how to use *ferret* to use and evaluate different gradient based approaches in the task of Natural Language Inference.

We will use`distilbert-base-uncased-finetuned-sst-2-english` as model checkpoint.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from ferret import (
    Benchmark,
    GradientExplainer,
    IntegratedGradientExplainer,
    LIMEExplainer,
    SHAPExplainer,
)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
premise = "I first thought that I liked the movie, but upon second thought it was actually disappointing."
hypothesis = "The movie was good."
sample = (premise, hypothesis)

In [4]:
model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.


In [5]:
ig = IntegratedGradientExplainer(model, tokenizer, multiply_by_inputs=True)
g = GradientExplainer(model, tokenizer, multiply_by_inputs=True)
l = LIMEExplainer(model, tokenizer)

No helper provided. Using default 'text-classification' helper.


In [6]:
bench = Benchmark(model, tokenizer, task_name="nli", explainers=[ig, g, l])

Overriding helper for explainer <ferret.explainers.gradient.IntegratedGradientExplainer object at 0x107526f50>
Overriding helper for explainer <ferret.explainers.gradient.GradientExplainer object at 0x28d55d660>
Overriding helper for explainer <ferret.explainers.lime.LIMEExplainer object at 0x107527d90>


You can use `targets` to see what are the available targets classes. You can then use them as integers (e.g., `2`) or in textual form (e.g., `contradiction`).

In [7]:
bench.targets

{0: 'entailment', 1: 'neutral', 2: 'contradiction'}

In [8]:
# get the prediction from our model
bench.score(sample)

{'entailment': 0.06558265537023544,
 'neutral': 0.17329996824264526,
 'contradiction': 0.7611173987388611}

In [9]:
# explain the contradiction class
exp = bench.explain(sample, target="contradiction")

Explainer:   0%|          | 0/3 [00:00<?, ?it/s]

Batch:   0%|          | 0/85 [00:00<?, ?it/s]

In [10]:
# show explanations
bench.show_table(exp)

,[CLS],▁I_0,▁first,▁thought_0,▁that,▁I_1,▁liked,▁the,▁movie_0,",",▁but,▁upon,▁second,▁thought_1,▁it,▁was_0,▁actually,▁disappointing,._0,[SEP]_0,▁The,▁movie_1,▁was_1,▁good,._1,[SEP]_1
Integrated Gradient (x Input),0.00,-0.14,0.26,-0.03,-0.01,0.05,-0.92,0.03,0.07,0.03,0.24,0.22,1.05,0.38,-0.39,-0.45,-0.45,0.15,0.39,-0.28,-0.00,-0.17,-0.55,-0.67,-0.21,0.00
Gradient (x Input),0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00
LIME,0.00,0.04,-0.03,0.10,0.04,-0.03,-0.07,-0.04,-0.00,0.01,0.03,0.03,-0.07,-0.03,0.03,0.00,0.01,0.32,-0.04,0.00,-0.02,-0.09,-0.06,0.12,0.04,0.00


In [11]:
# evaluate explanations and show faithfulness metrics
bench.show_evaluation_table(bench.evaluate_explanations(exp, target="contradiction"))

Explanation eval:   0%|          | 0/3 [00:00<?, ?it/s]

,aopc_compr,aopc_suff,taucorr_loo
Explainer,,,
Integrated Gradient (x Input),0.43,0.74,-0.11
Gradient (x Input),0.62,0.64,0.21
LIME,0.67,-0.15,0.65
